# Analysing a Log with LTL

The class `ProcessMiningTasks.ConformanceChecking.LTLAnalyzer.LTLAnalyzer` provides a way to check if the log conforms to a ltl formula. The formula can be a standard, we have a set of templates (including TB-Declare templates), or it can be a formula inputed by the user. This class makes use of two other classes `ProcessModels.LTLModel.LTLModel` and `ProcesModels.LTLModel.LTLModelTemplate`. The `LTLAnalyzer` class contains the following function:
1. `run` that returns a Pandas Dataframe. This Dataframe contains all traces in one column, in the second column we have the result of the conformance check, i.e., if a trace conforms to the LTL formula or not.

To use this class we need first to load a `xes` log and create a LTLModel object, which contains our formula.
The LTLModel can be created either using directly the `LTLModel` class or by using the `LTLModelTemplate` class.
We have to pass the name of a template to the constructor of the class`LTLModelTemplate`.

**NOTE:** For the grammar of LTL formulae we follow this model: https://marcofavorito.me/tl-grammars/v/7d9a17267fbf525d9a6a1beb92a46f05cf652db6/

In [4]:
import sys
import os
import pathlib

SCRIPT_DIR = pathlib.Path("..", "src").resolve()
sys.path.append(os.path.dirname(SCRIPT_DIR))

from src.Declare4Py.D4PyEventLog import D4PyEventLog

log_path = os.path.join("..", "tests", "Sepsis Cases.xes.gz")
event_log = D4PyEventLog()
event_log.parse_xes_log(log_path)

from src.Declare4Py.ProcessModels.LTLModel import LTLModel
from src.Declare4Py.ProcessModels.LTLModel import LTLModelTemplate

model = LTLModel()
model.parse_from_string("F(a)")
template = LTLModelTemplate('eventually_activity_a')


parsing log, completed traces ::   0%|          | 0/1050 [00:00<?, ?it/s]

/home/xhedj/.local/lib/python3.11/site-packages/pm4py/utils.py:486: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


In order to use the template object, we need to call `LTLModelTemplate`'s function `get_templ_model`, which expects a number of parameters equal to those expected by the template and returns an LTLModel object containing the parsed formula of the template.

In [5]:
template = LTLModelTemplate('eventually_activity_a')
model = template.get_templ_model('activity1')

Then we can create our `LTLAnalyzer` object and call the `run` function

In [6]:
from src.Declare4Py.ProcessMiningTasks.ConformanceChecking.LTLAnalyzer import LTLAnalyzer

analyzer = LTLAnalyzer(event_log, model)
analyzer.run()

Exception: the Lydia command lydia --logic=ltlf --inline=F(activityb) -p failed.
stdout=
stderr=docker: Got permission denied while trying to connect to the Docker daemon socket at unix:///var/run/docker.sock: Post "http://%2Fvar%2Frun%2Fdocker.sock/v1.24/containers/create": dial unix /var/run/docker.sock: connect: permission denied.
See 'docker run --help'.


## Using the LTLModel templates

### `eventually_activity_a`
Takes one activity as input as a string.
Formula:
- `F(activity)`

In [ ]:
template = LTLModelTemplate('eventually_activity_a')
model = template.get_templ_model('CRP')
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
print(data)

### `next_a`
Takes one activity as input.
Formula:
- `X(activity)`

In [ ]:
template = LTLModelTemplate('next_a')
model = template.get_templ_model('ER Triage')
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
print(data)

### `eventually_a_then_b`
Takes two activities as input.
Formula:
- `F(activity1 && F(activity2))`

In [ ]:
template = LTLModelTemplate('eventually_a_then_b')
model = template.get_templ_model('Leucocytes', 'CRP')
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
print(data)

### `eventually_a_or_b`
Takes two activities as input.
Formula:
- `F(activity1) || F(activity2)`

In [ ]:
template = LTLModelTemplate('eventually_a_then_b')
model = template.get_templ_model('Leucocytes', 'CRP')
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
print(data)

### `eventually_a_next_b`
Takes two activities as input. Formula:
- `F(activity1 && X(activity2))`

In [ ]:
template = LTLModelTemplate('eventually_a_next_b')
model = template.get_templ_model('Leucocytes', 'CRP')
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
print(data)

### `eventually_a_then_b_then_c`
Takes three activities as input. Formula:
- `F(activity1 && F(activity2 && F(activity3)))`

In [ ]:
template = LTLModelTemplate('eventually_a_then_b_then_c')
model = template.get_templ_model('ER Registration','Leucocytes', 'CRP')
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
print(data)

### `eventually_a_next_b_next_c`
Takes three activities as input. Formula:
- `F(activity1 && X(activity2 && X(activity3)))`

In [1]:
template = LTLModelTemplate('eventually_a_next_b_next_c')
model = template.get_templ_model('ER Registration', 'CRP', 'Leucocytes')
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
print(data)

NameError: name 'LTLModelTemplate' is not defined

## Branched Declare template models
A series of templates for branched declare. All functions take two lists of strings as parameters. The first parameter is the source while the second list is the target. They all return a string, which is the LTL formula.

### `responded existence`

In [ ]:
template = LTLModelTemplate('responded_existence')
source_list = ["ER Triage", "CRP", "Leucocytes"]
target_list = ["Leucocytes", "Admission NC", "Release A"]
model = template.get_templ_model(source_list, target_list)
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
print(data)

### `response`

In [ ]:
template = LTLModelTemplate('response')
source_list = ["ER Triage", "CRP", "Leucocytes"]
target_list = ["Leucocytes", "Admission NC", "Release A"]
model = template.get_templ_model(source_list, target_list)
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
print(data)

### `alternate_response`

In [ ]:
template = LTLModelTemplate('alternate_response')
source_list = ["ER Triage", "CRP", "Leucocytes"]
target_list = ["Leucocytes", "Admission NC", "Release A"]
model = template.get_templ_model(source_list, target_list)
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
print(data)

### `chain_response`

In [ ]:
template = LTLModelTemplate('chain_response')
source_list = ["ER Triage", "CRP", "Leucocytes"]
target_list = ["Leucocytes", "Admission NC", "Release A"]
model = template.get_templ_model(source_list, target_list)
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
print(data)

### `precedence`

In [ ]:
template = LTLModelTemplate('precedence')
source_list = ["ER Triage", "CRP", "Leucocytes"]
target_list = ["Leucocytes", "Admission NC", "Release A"]
model = template.get_templ_model(source_list, target_list)
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
print(data)

### `alternate_precedence`

In [ ]:
template = LTLModelTemplate('alternate_precedence')
source_list = ["ER Triage", "CRP", "Leucocytes"]
target_list = ["Leucocytes", "Admission NC", "Release A"]
model = template.get_templ_model(source_list, target_list)
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
print(data)

### `chain_precedence`

In [ ]:
template = LTLModelTemplate('chain_precedence')
source_list = ["ER Triage", "CRP", "Leucocytes"]
target_list = ["Leucocytes", "Admission NC", "Release A"]
model = template.get_templ_model(source_list, target_list)
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
print(data)

### `not_responded_existence`

In [ ]:
template = LTLModelTemplate('not_responded_existence')
source_list = ["ER Triage", "CRP", "Leucocytes"]
target_list = ["Leucocytes", "Admission NC", "Release A"]
model = template.get_templ_model(source_list, target_list)
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
print(data)

### `not_response`

In [ ]:
template = LTLModelTemplate('not_response')
source_list = ["ER Triage", "CRP", "Leucocytes"]
target_list = ["Leucocytes", "Admission NC", "Release A"]
model = template.get_templ_model(source_list, target_list)
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
print(data)

### `not_precedence`

In [ ]:
template = LTLModelTemplate('not_precedence')
source_list = ["ER Triage", "CRP", "Leucocytes"]
target_list = ["Leucocytes", "Admission NC", "Release A"]
model = template.get_templ_model(source_list, target_list)
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
print(data)

### `not_chain_response`

In [ ]:
template = LTLModelTemplate('not_chain_response')
source_list = ["ER Triage", "CRP", "Leucocytes"]
target_list = ["Leucocytes", "Admission NC", "Release A"]
model = template.get_templ_model(source_list, target_list)
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
print(data)

### `not_chain_precedence`

In [ ]:
template = LTLModelTemplate('not_chain_precedence')
source_list = ["ER Triage", "CRP", "Leucocytes"]
target_list = ["Leucocytes", "Admission NC", "Release A"]
model = template.get_templ_model(source_list, target_list)
analyzer = LTLAnalyzer(event_log, model)
data = analyzer.run()
print(data)